В данном обзоре мы рассмотрим два подхода к обучению языковых моделей на основе трансформеров:

- GPT (статья "[Improving Language Understanding by Generative Pre-Training](https://s3-us-west-2.amazonaws.com/openai-assets/research-covers/language-unsupervised/language_understanding_paper.pdf)", 2018). GPT в дальнейшем развитие в виде [GPT-2]($Language Models are Unsupervised Multitask Learners$) и далее [GPT-3]($Language Models are Few-Shot Learners$), которая стала способна генерировать тексты, [почти неотличимые от написанных человеком](https://twitter.com/raphamilliere/status/1289129723310886912).

- BERT (статья "[BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding](https://arxiv.org/abs/1810.04805)", 2018). Создание BERT многие упоминают как "ImageNet moment" в области NLP, проводя аналогию с созданием [AlexNet]($ImageNet Classification with Deep Convolutional Neural Networks$), давшей толчок развитию компьютерного зрения в 2012 году.

### Self-supervised learning в NLP

В большинстве задач, в том числе в задачах NLP (обработки естественного языка), неразмеченных данных доступно на порядки больше, чем размеченных. Неразмеченные данные в NLP - это любые тексты (из книг, Википедии, новостных сайтов и так далее). Можно собрать сотни гигабайт таких текстов. Общая идея подхода, называемого предобучением без учителя (unsupervised pretraining) в том, чтобы каким-то образом использовать для предобучения большое количество неразмеченных данных. Мы надеемся, что обучившись на таких данных, модель научится понимать структуру языка. Такую модель затем можно "дообучить" для решения какой-то конкретной задачи.

Технически нейронные сети требуют для обучения размеченные данные, то есть набор пар "вопрос + эталонный ответ". Каким же образом можно обучать их на неразмеченных данных? Для этого можно использовать подход, называемый self-supervised learning, то есть автоматическая генерация пар "вопрос + эталонный ответ" из неразмеченных данных. Сначала рассмотрим примеры не из области NLP:

- **Пример 1.** Автоэнкодеры. Выходные данные равны входным, то есть на выходе сеть должна выдать то же самое, что получила на вход. В качестве альтернативы на вход можно подавать искаженные данные, например изображение с наложенным шумом, а эталонным ответом (разметкой) считать исходное изображение.

- **Пример 2.** Задача in-painting в компьютерном зрении. На вход подается изображение с вырезанным участком (как в cutout), а на выходе должна выдать исходное изображение.

Теперь рассмотрим применение этого подхода на текстовых данных:

- **Вариант 1.** Мы выбираем из случайного текста случайный участок небольшого размера и удаляем в нем центральное слово. На вход модель получает участок без центрального слова, и на выходе должна определить, какое слово было удалено. Либо, наоборот, получив на вход центральное слово, нужно предсказать контекст.

- **Вариант 2.** *Masked lauguage modeling* - близкий аналог задачи in-painting. В тексте определенный процент слов заменяется на токен [MASK]. Сеть должна определить, какие слова стояли на месте [MASK]. Технически это реализуется таким образом: для каждого токена [MASK] сеть выдает распределение вероятностей по всем словам в словаре. Мы знаем удаленное слово, то есть имеем эталонный ответ. В качестве функции потерь можно применить категориальную кроссэнтропию между выданным распределением вероятностей и эталонным ответом. То есть данная задача решается как задача классификации.

- **Вариант 3.** *Next sentence prediction*. Входные данные составлены из двух предложений, соединенных токеном [SEP]. Сеть должна определить, стояли ли эти предложения друг за другом в исходном тексте, или же взяты из разных частей текста. То есть сеть решает задачу бинарной классификации.

- **Вариант 4.** Получив на вход незаконченный текст, сеть должна определить, какое слово стоит следующим.

Перечисленные задачи называются задачами языкового моделирования. Они имеют общую черту: верный ответ может быть далеко не единственным. Например, из текста вырезан участок, в котором пропущено имя. Если это имя больше не встречается в вырезанном участке, то сети неоткуда брать нужную информацию. Либо некое пропущенное слово можно заменить синонимом, и по смыслу такой ответ можно тоже считать верным. Но, тем не менее, метод self-supervised learning хорошо работает, поскольку используется очень много данных, и для минимизации loss модели нужно научиться угадывать среднестатистически наиболее вероятные ответы, что тоже является непростой задачей, требующей понимания языка.

Вариант 1 был реализован в модели [word2vec]($Efficient Estimation of Word Representations in Vector Space$) (2013). В word2vec для каждого слова в словаре обучается вектор-эмбеддинг. По сути это означает, что word2vec работает на уровне отдельных слов: модель выучивает среднестатистический смысл каждого слова. Но word2vec сам по себе не может обрабатывать фразы и длинные тексты и извлекать информацию о смысле всего текста целиком.

Идея обучения языковой модели, способной формировать высокоуровневые представления не отдельных слов, а целых фраз или текстов, является далеко не новой. Были попытки обучить языковую модель с помощью LSTM (см. [Dai et al.]($Semi-supervised Sequence Learning$) (2015), [Howard and Ruder]($Universal Language Model Fine-tuning for Text Classification$) (2018)). Однако LSTM, по-видимому, не слишком хорошо подходит для такой задачи. Несмотря на то, что сети [LSTM]($Long Short-term Memory$) (Hochreiter and Schmidhuber, 1997) были спроектированы как раз для противодействия "забыванию" предыдущей информации при поступлении новой, эта проблема все же сохраняется, как показано например в [этой недавней работе]($Slower is Better: Revisiting the Forgetting Mechanism in LSTM for Slower Information Decay$).

В 2017 году была изобретена архитектура "трансформер" (на которую я делал [обзор]($Attention Is All You Need$). Трансформеры способны намного эффективнее моделировать зависимости между отдаленными друг от друга участками текста. Приведем цитату из оригинальной работы по трансформерам:

> Learning long-range dependencies is a key challenge in many sequence transduction tasks. One key factor affecting the ability to learn such dependencies is the length of the paths forward and backward signals have to traverse in the network. The shorter these paths between any combination of positions in the input and output sequences, the easier it is to learn long-range dependencies.

Кроме того, важными особенностями трансформера является эффективная паралелизация обучения и существенный рост качества с ростом объема обучающего датасета и количества слоев в модели. Все это позволило обучать намного более эффективные языковые модели.

- **GPT** (сокращение от "Generative Pre-Training"), предложенная исследователями из OpenAI в 2018 году, использует для предобучения вариант 4 (из списка описанных выше), то есть предсказывает следующее слово в заданном тексте. В качестве архитектуры нейронной сети используется декодер трансформера.

- **BERT** (сокращение от "Bidirectional Encoder Representations from Transformers"), предложенная исследователями из Google в 2018 году, использует для предобучения вариант 2 (masked lauguage modeling) и вариант 3 (next sentence prediction). В качестве архитектуры нейронной сети используется энкодер трансформера.

Таким образом, GPT и BERT - это не архитектуры, а подходы к предобучению языковых моделей на неразмеченных данных.

### Модель GPT

Статья: [Improving Language Understanding by Generative Pre-Training](https://cdn.openai.com/research-covers/language-unsupervised/language_understanding_paper.pdf) (Radford et al., 2018)

GPT решает задачу предсказания следующего токена в последовательности на основе предыдущих токенов. В качестве токенов авторы предлагают использовать [Byte pair encoding]($Neural Machine Translation of Rare Words with Subword Units$), то есть целые слова и наиболее часто встречающиеся части слов. Способ токенизации может влиять на качество решения, но не влияет на общий подход к задаче.

В качестве архитектуры используется модификация декодера трансформера, в которой не используется информация из энкодера (энкодер при этом вообще отсутствует). По сравнению с энкодером трансформера, декодер работает авторегрессивно, то есть первые N входных токенов однозначно определяют первые N выходных токенов (подробнее см. [здесь]($Attention Is All You Need$), раздел "Маскирование в декодере"). Иначе говоря, N-й входной токен не влияет на выходные токены от 0-го до (N-1)-го.

При обучении вход модели подается последовательность из N токенов, взятых из случайного текста. Обработав эту последовательность, модель выдает N выходных векторов, которые затем линейным слоем с softmax преобразуются в распределение вероятностей по каждому слову в словаре. Модель должна предсказывать следующий токен, то есть *i-й элемент выходной последовательности является распределением вероятностей, argmax которого равен индексу (i+1)-го входного токена*. Соответственно, в качестве функции потерь по каждому токену можно использовать категориальную кроссэнтропию (logloss), и затем суммировать все полученные loss'ы. Важно, что если бы модель не была авторегрессивной (например, если бы мы использовали энкодер), то она могла бы напрямую использовать (i+1)-й входной токен для предсказания i-го выходного токена, и тогда ничему бы не научилась.

В целом схема обучения устроена точно так же, как в декодере трансформера из оригинальной работы ([Vaswani et al., 2017](https://arxiv.org/abs/1706.03762)). Разница лишь в отсутствии энкодера, токенов [START] и [END].

Обученная таким образом модель способна генерировать случайные тексты. На каждом шаге генерации мы получаем распределение вероятностей выходного токена, выбираем токен с максимальной вероятностью и добавляем его ко входной последовательности ("greedy search"). Можно использовать и более сложный вариант: [beam search](https://huggingface.co/blog/how-to-generate). Здесь все, опять таки, аналогично процедуре инференса трансформера из оригинальной работы.

### Дообучение модели GPT

Языковые модели, в том числе GPT и BERT, по задумке являются лишь "полуфабрикатами", которые нужно дообучать (fine-tuning) для решения конкретной задачи. При дообучении GPT выходной слой сети создается заново, чтобы выдавать ответ в требуемом формате. Формат может быть разный, в зависимости от типа задачи (классификация, регрессия, sequence-to-sequence и т. д.).

Авторы предлагают при дообучении (файн-тюнинге) минимизировать не только основной loss, соответствующий решаемой задаче, но также и дополнительный loss, который расчитывается тем же способом, что и в предобучении: по первым i входным токенам модель должна предсказывать (i+1)-й входной токен - уже на том датасете, на котором модель дообучается. Согласно авторам, добавление этого дополнительного loss'а улучшает качество файн-тюнинга, если файн-тюнинг выполняется на большом датасете, и практически никак не влияет при файн-тюнинге на маленьких датасетах.

Кроме того, если задача такова, что при файн-тюнинге требуются дополнительные токены (например [SEP]), то их эмбеддинги также нужно инициализировать случайными значениями перед тем, как начинать файн-тюнинг, поскольку при предобучении никаких дополнительных токенов не используется.

Авторы приводят несколько конкретных примеров задач:

<img src="assets/gpt.jpg" width="700" align="center">

**Классификация.** При файн-тюнинге GPT для задачи классификации в словарь добавляются два токена: [Start] и [Extract], эмбеддинги которых инициализируются случайными весами. Входной текст разбивается на токены, в начало добавляется токен [Start], в конец добавляется токен [Extract]. Также до начала файн-тюнинга создается дополнительный полносвязный слой-классификатор со случайными весами. Среди всех выходных векторов декодера выбирается вектор, соответствующий токену [Extract] (остальные отбрасываются). Выбранный вектор преобразуется слоем-классификатором с softmax в вероятности классов.

**Textual entailment (распознавание семантического следования)**. Задача бинарной классификации, в которой нужно определить, можно ли сделать вывод о верности второй фразы (hypothesis), если известно, что верна первая (premise). В этом случае входной последовательностью является конкатенация premise и hypothesis, между ними добавляется токен [delim]. Эмбеддинг этого токена также нужно инициализировать перед началом файн-тюнинга. В остальном технически эта задача сводится к предыдущей.

**Similarity (определение сходства двух фраз)**. В этом нет явного порядка на фразах (какаяиз фраз должна идти первой). Один из возможных подходов: подаем в модель сначала конкатенацию из первой и второй фразы, затем конактенацию из второй и первой фразы. Затем берутся вектора, полученные на токене [Extract] в обоих случаях, поэлементно складываются и обрабатываются полносвязным слоем с сигмоидой, выдающим степень сходства двух фраз. Конечно, в этом случае при обратном распространении ошибки градиенты по весам модели будут рассчитаны дважды - результаты просто складываются.

В **multiple choice** все приблизительно аналогично, из экономии места опустим этот пример в данном обзоре.

### Анализ эффективности GPT и режим zero-shot

Во-первых, авторы пробуют сразу обучать модель под конкретные задачи (без предобучения). В этом случае метрика качества получается существенно хуже. Тем самым доказывается, что предобучение позволяет повысить качество решения конкретных задач.

Важно также убедиться, что все слои в модели GPT обучаются так, чтобы играть некую полезную роль в получении ответа. Для этого при файн-тюнинге авторы пробуют отбрасывать часть последних слоев модели GPT. Авторы показывают, что чем больше слоев отбрасывается, тем ниже получается метрика качества при файн-тюнинге.

Авторы формулируют следующую гипотезу: предобучение эффективно потому, что в ходе предобучения модель неявно учится решать широкий спектр NLP-задач, чтобы научиться предсказывать следующее слово в различных текстах. Чтобы проверить эту гипотезу, авторы пробуют применять языковую модель GPT в режиме zero-shot.

**Решение задачи анализа тональности в режиме zero shot**

Например, мы хотим оценить качество модели GPT на задаче анализа тональности текста (позитивная/негативная). Для этой задачи существует размеченный датасет SST-2 из пар: фраза + тональность. Мы добавляем в начало каждой фразы слово "very" и обрабатываем эту фразу моделью (без файн-тюнинга на датасете SST-2). Хотя мы подаем на вход модели законченные фразы, но модель все равно пытается достроить текст, так как была обучена на этой задаче: она выдает распределение вероятностей для слова, которое должно идти следующим. Далее мы сравниваем вероятность слов "positive" и "negative". То слово, вероятность которого окажется выше, мы и считаем оценкой тональности.

Конечно, таким образом мы получим точность ниже, чем если бы делали файн-тюнинг на датасете SST-2. Но с другой стороны, так мы показываем, что модель способна решать некоторые задачи вообще без файн-тюнинга (в режиме zero-shot), пусть и неидеально.

**Решение задачи ответов на вопросы в режиме zero shot**

Рассмотрим также задачу ответов на вопросы (question answering) на датасете [RACE](https://paperswithcode.com/dataset/race) в режиме zero-shot с помощью GPT. Выберем один из вариантов ответа и подадим на вход модели конкатенацию контекста, вопроса и текущего варианта ответа. Пусть токен T входит в состав текущего варианта ответа. Посмотрим, какое распределение вероятностей модель выдала на предыдущем токене, и чему в нем равна вероятность токена T. Проделаем то же самое для все токенов текущего варианта ответа, и перемножим полученные вероятности (или сложим их логарифмы, что то же самое). Полученное число будет считать вероятностью данного варианта ответа. Таким образом получим вероятности для всех вариантов ответа (для N вариантов потребуется N запусков модели). Далее выберем вариант с наибольшей вероятностью.

**Оценка качества GPT в режиме zero shot**

В ходе предобучения GPT авторы периодически оценивали точность модели в режиме zero-shot на разных задачах. Для каждой задачи использовались свои эвристики. Выяснилось, что в ходе обучения на всех задачах точность растет. Это подтверждает гипотезу о том, что в ходе предобучения модель GPT учится решать различные NLP-задачи.

Ниже приведен график точности в режиме zero-shot на четырех задачах, в зависимости от длительности предобучения. При этом сравнивается трансформер и LSTM. Видно, что LSTM в режиме zero-shot на всех задачах работает хуже трансформера, и особенно плохо справляется с задачей ответов на вопросы, в которой требуется анализировать связи между частями текста, далеко отстоящими друг от друга.

<img src="assets/gpt_zero_shot2.jpg" width="400" align="center">

В дальнейшем выяснилось, что подход zero-shot очень эффективен. На некоторых задачах модель вообще не обязательно дообучать, достаточно лишь "дать понять", что именно вы хотите. Например, если добавить в конце текста "TL;DR" и дополнить этот текст с помощью GPT, то модель сгенерирует сокращенный вариант текста. Более подробно про это можно почитать, например, [здесь](https://andrewmayneblog.wordpress.com/2021/04/18/the-gpt-3-zero-shot-approach/).

<img src="assets/gpt_zero_shot.jpg" width="900" align="center">

Это достаточно просто объясняется. В ходе предобучения модели уже встречались такие тексты, где после "TL;DR" шел тот же текст в сокращенном варианте. Чтобы минимизировать ошибку на этих примерах, модель научилась решать задачу суммаризации.

### Модель BERT

Статья: [BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding](https://arxiv.org/abs/1810.04805) (Devlin et al., 2018)

TODO